Hikyuu pyecharts绘图
==============

1、引入交互式工具
-----------------
需从hikyuu.interactive引入，而不是直接从hikyuu库中引入（hikyuu是一个库，可用于编制其他的工具，而hikyuu.interactive.interactive是基于hikyuu库实现的交互式探索工具）

In [1]:
%matplotlib inline

%time from hikyuu.interactive import *
from hikyuu import *

# 设置绘图引擎为echarts
use_draw_engine('echarts')

Initialize hikyuu_2.6.0_202504301653_RELEASE_macosx_arm64 ...
2025-05-10 14:19:42.618 [HKU-I] - current python version: 3.10.15 (main.cpp:68)


2025-05-10 14:19:43,981 [INFO] runing in interactive session [<module>] (/Users/hy/.conda/envs/hikyuu/lib/python3.10/site-packages/hikyuu/__init__.py:127) [hikyuu::hku_info]
2025-05-10 14:19:43,982 [INFO] running in jupyter [<module>] (/Users/hy/.conda/envs/hikyuu/lib/python3.10/site-packages/hikyuu/__init__.py:134) [hikyuu::hku_info]


current plugin path: /Users/hy/.conda/envs/hikyuu/lib/python3.10/site-packages/hikyuu/plugin
2025-05-10 14:19:43.984 [HKU-I] - Using SQLITE3 BaseInfoDriver (BaseInfoDriver.cpp:57)
2025-05-10 14:19:43.985 [HKU-I] - Loading market information... (StockManager.cpp:492)
2025-05-10 14:19:43.986 [HKU-I] - Loading stock type information... (StockManager.cpp:510)
2025-05-10 14:19:43.986 [HKU-I] - Loading stock information... (StockManager.cpp:412)
2025-05-10 14:19:44.063 [HKU-I] - Loading stock weight... (StockManager.cpp:528)
2025-05-10 14:19:44.152 [HKU-I] - Loading block... (StockManager.cpp:120)
2025-05-10 14:19:44.215 [HKU-I] - Loading KData... (StockManager.cpp:124)
2025-05-10 14:19:44.216 [HKU-I] - Preloading day kdata to buffer (max: 100000)! (StockManager.cpp:170)
2025-05-10 14:19:44.217 [HKU-I] - 0.23s Loaded Data. (StockManager.cpp:132)
CPU times: user 1.46 s, sys: 318 ms, total: 1.77 s
Wall time: 1.71 s


2、创建交易系统并运行
--------------------

In [2]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(init_cash = 300000)

#创建信号指示器（相对强弱信号）
ind = RSI(C, 24)
my_sg = SG_Band(ind, 30, 70)

#固定每次买入1股
my_mm = MM_FixedCount(1)

#创建交易系统并运行
sys = SYS_Simple(tm = my_tm, sg = my_sg, mm = my_mm)
sys.set_param("buy_delay", False)
sys.set_param("sell_delay", False)
#日线信号
candle_num = 500
q = Query(-candle_num, constant.null_int64, Query.DAY)
symbol = 'sh000001'
sys.run(sm[symbol], q)

3、绘制曲线观察
---------------

In [3]:
chart = sys.plot()
k = sm[symbol].get_kdata(q)
c = CLOSE(k)
chart.render_notebook()

In [4]:
k.plot()

In [5]:
ind(c).plot(kref=k)

In [6]:
ind(c).bar(kref=k)

In [7]:
ind1 = TA_RSI(timeperiod=24)
ind2 = RSI(24)
chart = ind1(c).plot(kref=k)
ind2(c).plot(kref=k, chart=chart)
chart

TypeError: TA_RSI(): incompatible function arguments. The following argument types are supported:
    1. (n: int = 14) -> hikyuu.cpp.core310.Indicator
    2. (arg0: hikyuu.cpp.core310.IndParam) -> hikyuu.cpp.core310.Indicator
    3. (arg0: hikyuu.cpp.core310.Indicator, arg1: hikyuu.cpp.core310.IndParam) -> hikyuu.cpp.core310.Indicator
    4. (arg0: hikyuu.cpp.core310.Indicator, arg1: hikyuu.cpp.core310.Indicator) -> hikyuu.cpp.core310.Indicator
    5. (data: hikyuu.cpp.core310.Indicator, n: int = 14) -> hikyuu.cpp.core310.Indicator

Invoked with: kwargs: timeperiod=24

4、绘制资金收益曲线
---------------------

In [ ]:
#绘制资金收益曲线
x = my_tm.get_profit_curve(k.get_datetime_list(), q.ktype) # datetime is lost there
x = PRICELIST(x)
x.plot(kref=k).render_notebook()

5、回测统计报告
----------------------

In [ ]:
#回测统计
from datetime import datetime

per = Performance()
print(per.report(my_tm, Datetime(datetime.today())))
